# 📊 Experiment 03: Eigenvalue Spectrum Analysis

## Mathematical Proof of Expressivity Difference

While heatmaps are visually compelling, **eigenvalues are mathematically objective**. This notebook proves that quantum kernels have higher "effective rank" than classical RBF kernels.

---

### Key Concept: Eigenvalue Spectrum

The eigenvalue spectrum of a kernel matrix reveals:
- **Fast decay** → Low expressivity (compressed representation)
- **Slow decay** → High expressivity (rich representation)

### What We'll Prove:
> At 2 qubits, we observe a "Rank Bottleneck" — the quantum Hilbert space (4 dimensions) limits expressivity.

## Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.preprocessing import MinMaxScaler

# Import from our src modules
from src.classical import compute_rbf_kernel, compute_eigenspectrum, compute_effective_rank
from src.quantum import QuantumKernelComputer
from src.visualization import plot_spectrum_comparison

%matplotlib inline
print("✅ All imports successful!")

## Step 1: Generate Data (Same as Experiment 02)

In [ ]:
print("🧪 Regenerating Data for Consistency...")

X, y = make_circles(n_samples=20, factor=0.5, noise=0.05, random_state=42)

# Sort for consistency
sort_idx = np.argsort(y)
X = X[sort_idx]
y = y[sort_idx]

# Scale to [0, 2π]
scaler = MinMaxScaler(feature_range=(0, 2 * np.pi))
X_scaled = scaler.fit_transform(X)

print(f"📊 Data shape: {X_scaled.shape}")

## Step 2: Compute Kernel Matrices

In [ ]:
print("📐 Computing Classical Kernel...")
K_classical = compute_rbf_kernel(X_scaled, gamma=1.0)

print("⚛️ Computing Quantum Kernel (2 Qubits)...")
qkc = QuantumKernelComputer.from_zz_feature_map(num_features=2, reps=2, entanglement='linear')
K_quantum = qkc.compute_kernel_matrix(X_scaled)

print("✅ Both kernel matrices computed!")

## Step 3: Spectral Decomposition (The Math)

We extract eigenvalues from each kernel matrix:

$$K = V \Lambda V^T \quad \Rightarrow \quad \Lambda = \text{diag}(\lambda_1, \lambda_2, ..., \lambda_n)$$

In [ ]:
print("📊 Computing Eigenvalue Spectra...")

# Compute normalized eigenvalue spectra
spectrum_classical = compute_eigenspectrum(K_classical, normalize=True)
spectrum_quantum = compute_eigenspectrum(K_quantum, normalize=True)

print(f"\n📈 Classical RBF Spectrum:")
print(f"   Top 5 eigenvalues: {spectrum_classical[:5]}")
print(f"   Effective Rank (>1%): {compute_effective_rank(spectrum_classical)}")

print(f"\n📈 Quantum Fidelity Spectrum:")
print(f"   Top 5 eigenvalues: {spectrum_quantum[:5]}")
print(f"   Effective Rank (>1%): {compute_effective_rank(spectrum_quantum)}")

## Step 4: Visualize the Comparison

In [ ]:
print("🎨 Plotting Spectrum Comparison...")

fig = plot_spectrum_comparison(
    spectrum_classical,
    spectrum_quantum,
    classical_label="Classical RBF (Fast Decay)",
    quantum_label="Quantum Fidelity (High Rank)",
    title="Eigenvalue Spectrum: Expressivity Comparison",
    save_path="../results/figures/eigenvalue_spectrum.png"
)

plt.show()

## 📖 Interpretation: The 2-Qubit Bottleneck

### What We Observe:
At 2 qubits, the **quantum eigenvalues crash early** (around component 4-5).

### Why This Happens:
- 2 qubits → Hilbert space dimension = 2² = 4
- The quantum feature map can only access 4 dimensions
- After 4 components, no more "room" for information

### The Bottleneck:
> **Low-qubit quantum models act as dimensionality compressors.** They force data into a constrained subspace, limiting expressivity.

This explains why 2-qubit quantum kernels may not outperform classical RBF — the feature space is too small!

## 🔬 Deep Dive: Where's the Crossover?

Let's find the exact point where classical beats quantum:

In [ ]:
# Find crossover point
for i in range(min(len(spectrum_classical), len(spectrum_quantum))):
    if spectrum_classical[i] > spectrum_quantum[i]:
        print(f"⚠️ Classical exceeds Quantum at component {i}")
        print(f"   Classical: {spectrum_classical[i]:.6f}")
        print(f"   Quantum:   {spectrum_quantum[i]:.6f}")
        break
else:
    print("✅ Quantum stays higher throughout!")

# The mathematical insight
print(f"\n📐 Key Numbers:")
print(f"   Hilbert space dimension: 2^2 = 4")
print(f"   Number of samples: {len(X)}")
print(f"   Quantum effective rank: {compute_effective_rank(spectrum_quantum)}")
print(f"   Classical effective rank: {compute_effective_rank(spectrum_classical)}")

## 🚀 Next Steps

**Can we break through the bottleneck?**

→ **Notebook 04**: We increase to 8 qubits and observe "Rank Explosion"!

Prediction: With 2⁸ = 256 dimensional Hilbert space, quantum should dominate classical.